#### The goal of this script is to split up the intraSC values into the pre and post-twist segments and then set them up for comparison in an LMER

In [2]:
import numpy as np
import pandas as pd
import os
import tqdm

In [3]:
timepoints_cut_off = 20
timepoints_cut_off_end = 10
stim_end = 1107
HRF = 4
pre_twist = 552 
post_twist = 752 

node = 1
sub_corr_reint = np.load(f'../../../darkend/data/_multivariate_intraSC/intraSC_pattern_all_node{node}_pearsonr.npy',allow_pickle=True).item()

sub_list = []
r_val = [] #going to apply the z-transformed r-value to this!! 
node_list = []
label = []

nodes = list(range(1,101)) + ['hippocampus_L','hippocampus_R']
for node in tqdm.tqdm(nodes):
    sub_corr_reint = np.load(f'../../../darkend/data/_multivariate_intraSC/intraSC_pattern_all_node{node}_pearsonr.npy',allow_pickle=True).item()

    
    for sub in range(len(sub_corr_reint.keys())):
        
        pretwist_this_sub = sub_corr_reint[sub][timepoints_cut_off:pre_twist]
        posttwist_this_sub = sub_corr_reint[sub][post_twist:stim_end]
        pretwist_this_sub = [np.arctanh(r) for r in pretwist_this_sub]
        posttwist_this_sub = [np.arctanh(r) for r in posttwist_this_sub]
        
        
        sub_list.append(sub)
        r_val.append(np.median(pretwist_this_sub))
        label.append('pre')
        node_list.append(node)
        
        sub_list.append(sub)
        r_val.append(np.median(posttwist_this_sub))
        label.append('post')
        node_list.append(node) #for both pre and post twist
    



100%|██████████| 102/102 [00:08<00:00, 11.38it/s]


In [4]:

pre_post = pd.DataFrame()
pre_post['node'] = node_list
pre_post['sub'] = sub_list
pre_post['r'] = r_val
pre_post['label'] = label

pre_post

,node,sub,r,label
0,1,0,0.012219,pre
1,1,0,-0.013930,post
2,1,1,0.012914,pre
3,1,1,0.025541,post
4,1,2,0.022542,pre
...,...,...,...,...
7339,hippocampus_R,33,0.018886,post
7340,hippocampus_R,34,0.021091,pre
7341,hippocampus_R,34,0.006131,post
7342,hippocampus_R,35,-0.017821,pre


In [5]:
pre_post.to_csv('1.2._pre_post_for_LME_all_subcortical_new.csv')